# Projeto back-app

## Preparando o acesso ao Ambiente

In [ ]:
export KUBECONFIG=~/kubeconfig
export myNamespace=<seu namespace>
cd ~/work/inovacao
echo $PWD

1. Publicando o Projeto back-app

In [ ]:
helm upgrade --install --namespace $myNamespace --create-namespace back-app back-app/helm/back-app

2. Consultando os recursos do namespace

In [ ]:
kubectl get all -n $myNamespace

3. Validando o Serviço

    Para acessar o back-app fora do Cluster, necessitamos expor temporariamente o serviço
    
    Abra um terminal e execute o comando abaixo
```bash
export KUBECONFIG=~/kubeconfig
kubectl port-forward svc/back-app 8080:5000 -n <seu namespace> 
```

Consulte o serviço acessando o link abaixo:

In [ ]:
curl http://localhost:8080/health

Agora que os PODs estão em execução, podemos também validar a aplicação front-app.  


In [ ]:
echo http://kdop-learn.krthomolog.com.br/$myNamespace/front-app/health
echo http://kdop-learn.krthomolog.com.br/$myNamespace/front-app/index

### ConfigMap

Como o front-app acessa o back-app?

In [ ]:
kubectl describe cm/front-app -n $myNamespace

### HPA

```yaml
apiVersion: autoscaling/v2beta1
kind: HorizontalPodAutoscaler
metadata:
  name: back-app
  labels:
    back-app
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: back-app
  minReplicas: 1
  maxReplicas: 3
  metrics:
    - type: Resource
      resource:
        name: cpu
        targetAverageUtilization: 80
    - type: Resource
      resource:
        name: memory
        targetAverageUtilization: 80
```

In [ ]:
kubectl get hpa -n $myNamespace

### Volumes

O conceito de volume para PODs é o mesmo que para máquinas VM ou físicas, é uma área de armazenamento que será montada em um diretório ou em uma letra (windows).

![](media/aks-storage.png)

Fonte: [Opções de armazenamento para aplicativos no Serviço de Kubernetes do Azure (AKS)](https://docs.microsoft.com/pt-br/azure/aks/concepts-storage)

Existem muitos tipos de armazenamentos, os mais comuns são os efêmeros e os persistentes.

* [Efêmeros](https://kubernetes.io/docs/concepts/storage/ephemeral-volumes/): são os mais comuns e são perdidos quando o POD e reiniciado
* [Persistente](https://kubernetes.io/docs/concepts/storage/persistent-volumes/): Como o nome diz, são mantidos mesmo durante reinicio dos PODs. O Kubernetes encontra automaticamente os volumes correspondentes aos PODs.

Consulte [Kubernetes - Volumes](https://kubernetes.io/docs/concepts/storage/), para mais  opções. Além dos tipos, cada sistema oferece tipos diferentes de armazenamentos, tais como SSDs, discos mecânicos, etc. No kubernetes isso é representado pela classe de aramazenamento (Storage Class). Uma classe de armazenamento (SC) é utilizada para definir o tipo de volume persistente (PV) que será criado automaticamente a partir de um requerimento (PVC).

Para conhecer as classes de armazenamento disnponíveis no seu cluster, utilize o comando:

In [ ]:
kubectl get sc

Os nomes mudam de acordo com o lugar que o seu cluster está instalado, a lista acima são as classes da Azure.

Você pode omitir a classe de armazenamento, neste caso será utilizada a classe padrão.

A nossa aplicação é uma espécie de banco de dados rudimentar que precisa armazenar seus dados de forma persistente, para tal configuramos um requerimento de volume persistente.

Vejamos a configuração:

In [ ]:
helm template --namespace $myNamespace --create-namespace back-app ./back-app

Selecionamos o trecho referente ao volume:

```yaml
---
# Source: back-app/templates/pcv.yaml
kind: PersistentVolumeClaim
metadata:
  name: back-app-managed-disk
spec:
  accessModes:
  - ReadWriteOnce
  storageClassName: managed-premium
  resources:
    requests:
      storage: 1Gi
---
# Source: back-app/templates/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: back-app
  labels:
    helm.sh/chart: back-app-0.1.0
    app.kubernetes.io/name: back-app
    app.kubernetes.io/instance: back-app
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
spec:
  replicas: 1
  selector:
    matchLabels:
      app.kubernetes.io/name: back-app
      app.kubernetes.io/instance: back-app
  template:
    metadata:
      labels:
        app.kubernetes.io/name: back-app
        app.kubernetes.io/instance: back-app
    spec:
      serviceAccountName: back-app
      securityContext:
        {}
      containers:
        - name: back-app
          securityContext:
            {}
          image: "kdop/back-app:0.0.1"
          imagePullPolicy: Always
          ports:
            - name: http
              containerPort: 5000
              protocol: TCP
          livenessProbe:
            httpGet:
              path: /health
              port: http
          readinessProbe:
            httpGet:
              path: /health
              port: http
          resources:
            limits:
              cpu: 200m
              memory: 256Mi
            requests:
              cpu: 200m
              memory: 256Mi
          volumeMounts:
          - mountPath: "/tmp/inovacao/back-app"
            name: back-volume
      volumes:
        - name: back-volume
          persistentVolumeClaim:
            claimName: back-app-managed-disk
```

Um `PersistentVolume` foi criado dinamicamente em resposta ao nosso `PersistentVolumeClaim`. O K8s interage com as APIs da Azure para criar o PV na forma de um disco.

Você pode criar seus `PersistentVolume` e/ou associá-los a discos já existentes.

In [ ]:
kubectl get pv

O comando `get pv` listou um `PersistentVolumes` associado ao seu requerimento.

Na figura abaixo, você pode ver os dois discos na Azure.

![](media/azure-disks.png)

Se você descrever o `PersistentVolume` associado verá alguns labels que descrevem sua origem.

In [ ]:
kubectl describe pv/pvc-036a03c7-ae50-48e8-8952-8b9d7fbdf1b0 

Um olhar mais detalhado no disco na Azure revelará como o Kubernetes faz sua mágica para encontrar seus volumes e seus respectivos donos.

![](media/azure-disk.png)

Alguns desses labels também aparecem no `PersistentVolumeClaim`.

In [ ]:
kubectl describe pvc/back-app-managed-disk -n $myNamespace

[<img align="left" src="media/voltar.png" width="100"/>](03_Helm.ipynb) [<img align="right" src="media/avancar.png" width="100"/>](05_Cert.ipynb)